# LDA MODEL USING GENSIM LIBRARY 
Starting from the initial dataset, preprocess it and then apply the LDA model of gensim library. 
Finally, having the output, perform a binary classification analysis

## import libraries

In [1]:
!pip install pyLDAvis &> /dev/null

In [2]:
import numpy as np
import pandas as pd

import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [3]:
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
from nltk.corpus import stopwords
import re
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)

In [4]:
import pickle
from pprint import pprint
import collections
import numpy 
import matplotlib.pyplot as plt
import operator
import array
import pandas as pd


# Natural Language Tool Kit (NLTK) imports
import nltk
from nltk.data  import load
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Machine Learning Library (sklearn) imports
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer , CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

from gensim.models import TfidfModel

In [5]:
# Install nltk, Library suited for text mining and texts processing
import nltk
nltk.download('punkt') # containing the tokeinzers
import nltk
nltk.download('stopwords')  # Frequent or meaningless words of,an,a...

# Split a text into sentences
sentence_splitter = load('tokenizers/punkt/english.pickle')

# Split a sentence into words
tokenizer = TreebankWordTokenizer()

# Reduce to word to the root
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [6]:
punctuation = set([",", ".", ";", "/", ":", "-", "--" ,"!", "?", "(", ")","'",'"',"''", "``"])

# Set of common english words
stopwords_set = set(stopwords.words('english'))  # very common words in english
 

## text processing functions

In [7]:
# Preprocess a given text
def preprocess_text(review):
    tokens = []
    # A. Split into sentences
    for sentence in sentence_splitter.tokenize(review):
        # B. Split into tokens
        for token in tokenizer.tokenize(sentence):
            token = token.lower()
            # C. Filter on stoplist and punctuation
            if token not in stopwords_set and token not in punctuation:
                # D. Stemming (takes root)
                stem = stemmer.stem(token)
                tokens.append(stem)
    return tokens

## Import of datasets

Dataset sul drive condiviso: positive_text.review e negative_text.review

In [13]:
# Import the set of positive book reviews
with open('/content/gdrive/My Drive/positive_text.review', 'r') as myfile: #positive reviwes about books
    pos_books = myfile.readlines()


# Import the set of negative book reviews
with open('/content/gdrive/My Drive/negative_text.review', 'r') as myfile: #negative reviwes about books
    neg_books = myfile.readlines()


In [14]:
print("pos_books length : " , len(pos_books) , "\n")
print("neg_books length : " , len(neg_books))

pos_books length :  1000 

neg_books length :  1000


## preprocessing

In [15]:
# Dictionary of texts
B_pos={}
B_neg={}

# All possible words
features=[]

for i in range(1000):
  # Preprocess the i-th text
  pre_pos=preprocess_text(pos_books[i].replace("'",' '))
  pre_neg=preprocess_text(neg_books[i].replace("'",' '))

  # Add the text to the dictionary
  B_pos[i]=pre_pos
  B_neg[i]=pre_neg

  # Add all the tokens to the list
  for j in pre_pos:
    features.append(j)
  for j in pre_neg:
    features.append(j)

# list to set 
features=set(features)

# set to dictionary
feat={}

k=0
# Each word in features is associated to an integer (index of columns)
for i in features:
   feat[i]=k
   k+=1

In [16]:
a = list(B_pos.values())
b = list(B_neg.values())

data = a + b
len(data)

2000

## lda model

In [17]:
import gensim
# Create the bigram model using the Phrases class
bigram_model = gensim.models.Phrases(data, min_count=3, threshold=20)
# Create a Phraser object from the bigram model
bigram_phraser = gensim.models.phrases.Phraser(bigram_model)
# Apply the bigram model to the tokens
bigrams = [bigram_phraser[token] for token in data]
# Create the trigram model using the Phrases class
trigram_model = gensim.models.Phrases(bigrams, min_count=3, threshold=20)
# Create a Phraser object from the trigram model
trigram_phraser = gensim.models.phrases.Phraser(trigram_model)
# Apply the trigram model to the bigrams
trigrams = [trigram_phraser[bigram] for bigram in bigrams]

INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 158002 word types from a corpus of 177333 words (unigram + bigrams) and 2000 sentences
INFO:gensim.models.phrases:using 158002 counts as vocab in Phrases<0 vocab, min_count=3, threshold=20, max_vocab_size=40000000>
INFO:gensim.models.phrases:source_vocab length 158002
INFO:gensim.models.phrases:Phraser built with 652 phrasegrams
INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 159286 word types from a corpus of 172620 words (unigram + bigrams) and 2000 sentences
INFO:gensim.models.phrases:using 159286 counts as vocab in Phrases<0 vocab, min_count=3, threshold=20, max_vocab_size=40000000>
INFO:gensim.models.phrases:source_vocab length 159286
INFO:gensim.models.p

In [18]:
print(trigrams)

[['sphere', 'michael', 'crichton', 'excel', 'novel', 'certainli', 'hardest', 'put', 'crichton_novel', 'read', 'stori', 'revolv_around', 'man', 'name', 'norman', 'johnson', 'johnson', 'phycologist', 'travel', '4', 'civilan', 'remot', 'locat', 'pacif', 'ocean', 'help', 'navi', 'top', 'secret', 'misssion', 'quickli', 'learn', 'ocean', 'half', 'mile', 'long', 'spaceship', 'civilan', 'travel', 'center', '1000', 'feet', 'ocean', 'live', 'research', 'spacecraft', 'join', '5', 'navi', 'personel', 'help', 'run', 'oper', 'howev', 'surfac', 'typhoon', 'come', 'support', 'ship', 'surfac', 'must', 'leav', 'team', 'ten', 'stuck', '1000', 'feet', 'surfac', 'ocean', 'day', 'sea', 'find', 'spacecraft', 'actual', 'american', 'ship', 'explor', 'black', 'hole', 'brought', 'back', 'strang', 'thing', 'back', 'earth', 'novel', 'research', 'crichton_novel', 'still', 'lot', 'inform', 'random', 'thing', 'law', 'partial', 'pressur', 'behavior', 'analysi', 'would', 'strongli_recommend', 'book'], ['dr.', 'oz', 'ac

In [19]:
to_remove = ['kagan', 'christian']

In [20]:
trigrams_new = [[word for word in doc if word.lower() not in to_remove] for doc in trigrams]

In [21]:
dct = corpora.Dictionary(trigrams_new)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(17730 unique tokens: ['1000', '4', '5', 'actual', 'american']...) from 2000 documents (total 172073 corpus positions)


In [22]:
corpus = [dct.doc2bow(line) for line in trigrams_new]

In [23]:
tfidf = TfidfModel(corpus, id2word=dct)

low_value = 0.01
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
  bow = corpus[i]
  low_value_words = []
  tfidf_ids = [id for id, value in tfidf[bow]]
  bow_ids = [id for id, value in bow]
  low_value_words = [id for id, value in tfidf[bow] if value < low_value]
  drops = low_value_words+words_missing_in_tfidf
  for item in drops:
    words.append(dct[item])
  words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

  new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
  corpus[i] = new_bow

INFO:gensim.models.tfidfmodel:collecting document frequencies
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #0
INFO:gensim.models.tfidfmodel:calculating IDF weights for 2000 documents and 17729 features (134235 matrix non-zeros)


In [24]:
n_topics = 10

In [25]:
lda_model = LdaMulticore(corpus=corpus,
                         id2word=dct,
                         random_state=100,
                         num_topics=n_topics,
                         passes=10,
                         chunksize=1000,
                         batch=False,
                         alpha='symmetric',
                         decay=0.5,
                         offset=1.0,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.1
INFO:gensim.models.ldamodel:using symmetric eta at 0.1
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 10 topics, 10 passes over the supplied corpus of 2000 documents, updating every 1000 documents, evaluating every ~0 documents, iterating 100x with a convergence threshold of 0.001000
INFO:gensim.models.ldamulticore:training LDA model using 1 processes
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #1000/2000, outstanding queue size 1
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #1 = documents up to #2000/2000, outstanding queue size 2
INFO:gensim.models.ldamodel:merging changes from 1000 documents into a model of 2000 documents
INFO:gensim.models.ldamodel:topic #3 (0.100): 0.008*"one" + 0.006*"work" + 0.005*"book" + 0.004*"read" + 0.004*"peopl" + 0.003*"make" + 0.003*"love" + 0.003*"would"

In [26]:
# lda_ens = EnsembleLda(corpus=corpus,
#                       id2word=dct,
#                       num_models=4,
#                       num_topics=n_topics,)

In [27]:
#visualize LDA
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, corpus, dct, mds = 'mmds', R = 30)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.037338  0.042600       1        1  22.482975
8      0.025254  0.005973       2        1  19.750568
9      0.001323  0.131007       3        1  12.314590
7     -0.080559 -0.134326       4        1   8.624221
1     -0.110976  0.093178       5        1   7.073317
6      0.097846  0.107564       6        1   7.002222
0      0.109141 -0.113832       7        1   6.694269
5     -0.151646 -0.031653       8        1   6.385707
3      0.145526  0.009736       9        1   5.324914
4      0.001429 -0.110246      10        1   4.347217, topic_info=      Term         Freq        Total Category  logprob  loglift
9     book  3798.000000  3798.000000  Default  30.0000  30.0000
538    ...   483.000000   483.000000  Default  29.0000  29.0000
619  recip   119.000000   119.000000  Default  28.0000  28.0000
765    god   165.000000   165.000000  Default  27.0000  27.0000
69   stori   690.000000   690.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
719  could    16.136164   386.669289  Topic10  -6.1297  -0.0409
910  peopl    16.925005   539.607301  Topic10  -6.0819  -0.3264
227   know    15.746087   410.648273  Topic10  -6.1541  -0.1255
452  found    15.215705   336.722532  Topic10  -6.1884   0.0387
221     go    14.987328   402.897104  Topic10  -6.2035  -0.1558

[814 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
15958      7  0.977209      +
538        1  0.196312    ...
538        2  0.101256    ...
538        3  0.123987    ...
538        4  0.059927    ...
...      ...       ...    ...
1980       1  0.933510  yeast
7145       1  0.814687   zero
7145       2  0.062668   zero
7145       9  0.062668   zero
14081      9  0.798326  zukav

[2060 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 10, 8, 2, 7, 1, 6, 4, 5])

In [ ]:
# pyLDAvis.enable_notebook()
# pyLDAvis.gensim_models.prepare(lda_ens, corpus, dct, mds = 'mmds', R = 30)

In [28]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

INFO:gensim.models.ldamodel:topic #0 (0.100): 0.019*"book" + 0.009*"like" + 0.007*"recip" + 0.005*"one" + 0.005*"read" + 0.004*"also" + 0.004*"use" + 0.004*"time" + 0.004*"..." + 0.004*"cook"
INFO:gensim.models.ldamodel:topic #1 (0.100): 0.016*"book" + 0.006*"like" + 0.006*"one" + 0.005*"stori" + 0.005*"read" + 0.005*"even" + 0.004*"author" + 0.004*"charact" + 0.004*"time" + 0.004*"think"
INFO:gensim.models.ldamodel:topic #2 (0.100): 0.050*"book" + 0.015*"read" + 0.009*"one" + 0.006*"get" + 0.006*"like" + 0.006*"author" + 0.005*"good" + 0.005*"would" + 0.005*"much" + 0.004*"use"
INFO:gensim.models.ldamodel:topic #3 (0.100): 0.007*"god" + 0.007*"one" + 0.006*"book" + 0.004*"love" + 0.004*"..." + 0.004*"work" + 0.004*"read" + 0.004*"like" + 0.004*"thing" + 0.003*"us"
INFO:gensim.models.ldamodel:topic #4 (0.100): 0.019*"book" + 0.014*"..." + 0.009*"read" + 0.007*"one" + 0.006*"time" + 0.005*"dog" + 0.005*"write" + 0.005*"use" + 0.004*"stori" + 0.004*"interest"
INFO:gensim.models.ldamodel:

[(0,
  '0.019*"book" + 0.009*"like" + 0.007*"recip" + 0.005*"one" + 0.005*"read" + '
  '0.004*"also" + 0.004*"use" + 0.004*"time" + 0.004*"..." + 0.004*"cook"'),
 (1,
  '0.016*"book" + 0.006*"like" + 0.006*"one" + 0.005*"stori" + 0.005*"read" + '
  '0.005*"even" + 0.004*"author" + 0.004*"charact" + 0.004*"time" + '
  '0.004*"think"'),
 (2,
  '0.050*"book" + 0.015*"read" + 0.009*"one" + 0.006*"get" + 0.006*"like" + '
  '0.006*"author" + 0.005*"good" + 0.005*"would" + 0.005*"much" + 0.004*"use"'),
 (3,
  '0.007*"god" + 0.007*"one" + 0.006*"book" + 0.004*"love" + 0.004*"..." + '
  '0.004*"work" + 0.004*"read" + 0.004*"like" + 0.004*"thing" + 0.003*"us"'),
 (4,
  '0.019*"book" + 0.014*"..." + 0.009*"read" + 0.007*"one" + 0.006*"time" + '
  '0.005*"dog" + 0.005*"write" + 0.005*"use" + 0.004*"stori" + '
  '0.004*"interest"'),
 (5,
  '0.007*"peopl" + 0.006*"use" + 0.006*"one" + 0.005*"book" + 0.004*"would" + '
  '0.004*"good" + 0.003*"read" + 0.003*"diet" + 0.003*"mani" + 0.003*"believ"'),
 (

In [29]:
topic_distribution = lda_model.get_document_topics(corpus,minimum_probability=0)

In [30]:
np.shape(topic_distribution)
topic_distribution[0][0][1]

0.0009903318

In [31]:
x = np.array(topic_distribution)
X = x[:, :, 1]

neg = np.zeros(1000)
pos = np.ones(1000)
y = np.concatenate([pos,neg])

In [32]:
np.shape(y)

(2000,)

# GP REGRESSION

In [33]:
!pip install GPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.4/959.4 KB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPy: filename=GPy-1.10.0-cp38-cp38-linux_x86_64.whl size=3424794 sha256=d150858d0d4bbf7c1d9f7b0c9980dfa33a8f5f56ac68f894c8a81cb8c3e73140
  Stored in directory: /root/.cache/pip/wheels/48/b3/22/31f07cfd7b182ea3703151b7e5a7d6447e3e1ac6aa5c529413
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102566 sha256=c1b279a5165292a988253acdb1fae68e517726f663b741151873472207cfa6cf
  Stored in directory: /root/.cache/pip/wheels/66/78/6c/d98cb437834de5e29381786b4ba8a77ea68cca74653ab62713
Successfully built GPy paramz


In [34]:
import GPy
import numpy as np

# Load the preprocessed text data and the topic distributions
X = X # shape: (n_samples, n_topics)
Y = y # shape: (n_samples, n_outputs)

n_outputs = 1
Y = Y.reshape(-1, n_outputs)

# Binarize the labels to two classes
y = (y == 0).astype(int)

# Define the Gaussian Process classifier
kernel = GPy.kern.RBF(input_dim=n_topics)
model = GPy.models.GPClassification(X, Y , kernel=kernel)

# Train the GP model
model.optimize()



INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters


In [ ]:
y_pred, _ = model.predict(X)
accuracy = np.mean(np.round(y_pred) == Y)
accuracy


0.6775

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y, np.round(y_pred))
cm

array([[741, 259],
       [386, 614]])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(Y, np.round(y_pred), average='binary')
print(f1_score)
print(precision)
print(recall)

0.6556326748531768
0.7033218785796106
0.614
